In [1]:
import numpy as np

def analyze_experiment_trials(trials):
    training_SNR_dict = {'num_trials': 0,
                         'symbols_sent': [],
                         '3db_off': [0.0],
                         '5db_off': [0.0],
                         'max_num_logs': 0
                        }
    for trial_id, trial in trials.items():
        meta, result_array = trial['meta'], trial['data']
        train_SNR_db = meta['train_SNR_db']
        batch_size = meta['batch_size']
        num_results = meta['num_results']
        test_SNR_dbs = meta.get('test_SNR_dbs', result_array[0]['test_SNR_dbs'])
        if training_SNR_dict.get('test_SNR_dbs', None) is None:
            training_SNR_dict['SNR_db_off_for']= test_SNR_dbs[4]
            training_SNR_dict['test_SNR_dbs']=test_SNR_dbs
        else:
            assert training_SNR_dict['test_SNR_dbs']== test_SNR_dbs
            assert training_SNR_dict['SNR_db_off_for']== test_SNR_dbs[4]
        training_SNR_dict['num_trials'] += 1
        training_SNR_dict['max_num_logs'] = max(training_SNR_dict['max_num_logs'], num_results)
        # In order to deal with early stopping, we subtract if didn't converge,
        # if converged early, then we don't have to fill the array
        db5off = np.ones(training_SNR_dict['max_num_logs'])
        db3off = np.ones(training_SNR_dict['max_num_logs'])
        symbols_sent = training_SNR_dict['symbols_sent']
        for i, result in enumerate(result_array):
            assert i >= len(symbols_sent) or (result['batches_sent'] * batch_size) == symbols_sent[i]
            symbols_sent += [result['batches_sent'] * batch_size] if i >= len(symbols_sent) else []
            db_off = result['db_off'][4]
            if db_off > 5.0:
                db5off[i] -= 1.0
            if db_off > 3.0:
                db3off[i] -= 1.0
        training_SNR_dict['symbols_sent'] = symbols_sent
        # Below handles early stopping
        t5off = training_SNR_dict['5db_off']
        t3off = training_SNR_dict['3db_off']
        for j in range(len(t5off), training_SNR_dict['max_num_logs']):
            t5off = np.append(t5off, t5off[-1])
            t3off = np.append(t3off, t3off[-1])
        training_SNR_dict['5db_off'] = t5off + db5off
        training_SNR_dict['3db_off'] = t3off + db3off
        
        trial_final_bers = np.array(result_array[-1]['test_bers'])
        trial_final_bers = trial_final_bers[..., np.newaxis]
        if training_SNR_dict.get('final_bers', None) is None:
            training_SNR_dict['final_bers'] = trial_final_bers
        else:
            training_SNR_dict['final_bers'] = np.append(training_SNR_dict['final_bers'], trial_final_bers, axis=1)
    num_trials = training_SNR_dict['num_trials']
    training_SNR_dict['5db_off'] = training_SNR_dict['5db_off'] / num_trials
    training_SNR_dict['3db_off'] = training_SNR_dict['3db_off'] / num_trials
    sorted_bers = np.sort(training_SNR_dict['final_bers'], axis=1)
    training_SNR_dict['BER_low'] = sorted_bers[:, num_trials * 10 // 100]
    training_SNR_dict['BER_mid'] = sorted_bers[:, num_trials // 2]
    training_SNR_dict['BER_high'] = sorted_bers[:, num_trials * 90 // 100]
    return training_SNR_dict

In [2]:
import google.datalab.storage as storage
bucket_name='torch-echo'
bucket = storage.Bucket(bucket_name)
assert bucket.exists()

to_process =[]

for obj in bucket.objects():
    if ".npy" in obj.key and \
        "processed/" in obj.key:
        to_process += [obj]


In [3]:
from io import BytesIO, StringIO
# print([o.key for o in to_process])
for trials_npy in to_process:
  identifier = trials_npy.key.split("/")[-1].split(".")[0]
  uri = trials_npy.uri
  %gcs read --object $uri --variable data
  trials = np.load(BytesIO(data)).item()
  analysis = analyze_experiment_trials(trials)
  np.save(identifier+".npy", analysis)
  !gsutil -m mv -p  ./*.npy gs://torch-echo/analyzed/

Copying file://./gradient_passing-QPSK_classic_and_neural-13.npy [Content-Type=application/octet-stream]...
Copying file://./loss_passing-QPSK_neural_and_neural-13.npy [Content-Type=application/octet-stream]...
Removing file://./gradient_passing-QPSK_classic_and_neural-13.npy...            
Removing file://./loss_passing-QPSK_neural_and_neural-13.npy...                 
/ [2/2 files][  9.6 KiB/  9.6 KiB] 100% Done                                    
Operation completed over 2 objects/9.6 KiB.                                      
Copying file://./gradient_passing-QPSK_classic_and_neural-4_2.npy [Content-Type=application/octet-stream]...
Removing file://./gradient_passing-QPSK_classic_and_neural-4_2.npy...           
/ [1/1 files][  4.9 KiB/  4.9 KiB] 100% Done                                    
Operation completed over 1 objects/4.9 KiB.                                      
Copying file://./gradient_passing-QPSK_classic_and_neural-8_4.npy [Content-Type=application/octet-stream]...
Re

Copying file://./private_preamble-QPSK_neural_vs_poly-8_4.npy [Content-Type=application/octet-stream]...
Removing file://./private_preamble-QPSK_neural_vs_poly-8_4.npy...               
/ [1/1 files][  5.1 KiB/  5.1 KiB] 100% Done                                    
Operation completed over 1 objects/5.1 KiB.                                      
Copying file://./private_preamble-QPSK_neural_vs_selfalien-13.npy [Content-Type=application/octet-stream]...
Removing file://./private_preamble-QPSK_neural_vs_selfalien-13.npy...           
/ [1/1 files][  5.0 KiB/  5.0 KiB] 100% Done                                    
Operation completed over 1 objects/5.0 KiB.                                      
Copying file://./private_preamble-QPSK_neural_vs_selfalien-4_2.npy [Content-Type=application/octet-stream]...
Removing file://./private_preamble-QPSK_neural_vs_selfalien-4_2.npy...          
/ [1/1 files][  5.0 KiB/  5.0 KiB] 100% Done                                    
Operation completed over 1

In [14]:
test = np.load(identifier+".npy").item()
test.keys()

dict_keys(['SNR_db_off_for', 'final_bers', 'symbols_sent', 'test_SNR_dbs', 'max_num_logs', '3db_off', 'num_trials', 'BER_high', 'BER_low', 'BER_mid', '5db_off'])

In [ ]:
# gsutil -m cp -p  gs://torch-echo/analyzed/*.npy ./analysis/results/